# Photon Number Estimates

### This notebook calculates the estimated signal of Next-like detectors 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Constants in all cases
E_qbb = 2400. # keV
E_kr = 41.5 # keV
dEdx_qbb_blob = 20 # keV/cm
dEdx_qbb_track = 7 # keV/cm
dEdx_kr = 20 # no idea if this is true or not...

In [2]:
def N_i(E, W_i=22.4):
    """
    Returns number of ionization electrons released 
    in a material with an ionization energy of W_i
    with an energy of E.
    
    Input: 
        E - energy in keV
        W_i - ionization energy in eV, default to GXe
    """
    return E*1000./W_i

In [3]:
print('N_i(Qbb) = '+str(int(N_i(E_qbb)))+', N_i(kr) = '+str(int(N_i(E_kr))))

N_i(Qbb) = 107142, N_i(kr) = 1852


In [4]:
def SAF_Plane(r_TP, d_EL, d_A):
    """
    Solid angle fraction of the circular plane a distance 
    d_EL+d_A from the source.
    
    Input: r_TP - radius of tracking plane [mm]
           d_EL - length of EL gap [mm]
           d_A - distance between anode and SiPMs [mm]
    """
    return 2*np.pi * (1/(4*np.pi)) * (1 - ((d_EL+d_A)/2.)/(np.sqrt(r_TP**2. + ((d_EL+d_A)/2.)**2.)))

In [5]:
def SAF_SiPM(s, d_EL, d_A, sipm_width=2.):
    """
    Solid angle fraction of a SiPM on 
    the tracking plane.
    Input: s - size of SiPM [mm]
           d_EL - length of EL gap [mm]
           d_A - distance between anode and SiPMs [mm]
    """
    d = d_EL/2. + d_A - sipm_width
    return SAF_square(s, d)

def SAF_square(s, d):
    """
    Solid angle fraction of square plane a distance
    d from the source
    Reference: https://vixra.org/pdf/2001.0603v1.pdf
    
    Input: s - size of SiPM [mm]
           d - distance [mm]
    """
    num = s**2.
    denom = 4.*d**2. * np.sqrt(1. + 2.*s**2./(4.*d**2.))
    return (1./np.pi) * np.arctan( num / denom)

def SAF_square_approx(s, d):
    """
    Solid angle fraction of square plane, approx for small s 
    a distance d from the source.
    
    Input: s - size of SiPM [mm]
           d - distance from source [mm]
    """
    return s**2. / (4*np.pi*d**2.)

In [6]:
def Coverage(r_TP, s, N_sipms):
    """
    Coverage of tracking plane
    
    Input: r_TP - radius of tracking plane [mm]
           s - size of SiPM [mm]
           N_sipms - number of SiPMs [mm]
    """
    return s**2. * N_sipms / (np.pi * r_TP**2.)

In [7]:
def EL_yield_old(d_EL, Efield, P):    
    red_efield = Efield/(P)
    red_ELyield = 151*red_efield-116
    ELyield = red_ELyield*(d_EL/10.)*P
    return ELyield

def EL_yield(d_EL, Efield, P):
    """
    Returns the EL yeild, the number of photons per ionization
    electron crossing the EL gap.
    Reference: E.D.C.Freitas 2010, 'Secondary scintillation 
        yield in high-pressure xenon gas for neutrinoless double 
        beta decay  search'
        
    Input: d_EL - length of EL gap [mm], 
           Efield - electric field [kV/cm]
           P - pressure [bar]
    """
    b = 116. # / (bar*cm)
    a = 140. # / kV
    if P >= 8.:
        a = 170.
    elif P >= 6.:
        a = 161.
    elif P >= 5.:
        a = 151.
    elif P >= 4.:
        a = 142.
    elif P >= 2.:
        a = 141.
        
    ELyield = (a*Efield/P - b) * P*d_EL/10.
    
    return ELyield

In [8]:
def N_d(E, eta, epsilon, f):
    """
    Returns the number of photons detected
    
    Inputs: E - energy of event [keV]
            eta - gain of EL gap
            epsilon - efficiencies, 
            f - fraction of light to planes (Solid angle fraction * Coverage)
    """
    return N_i(E) * eta * epsilon * f

In [9]:
def dN_dt(dE_dx, v_d, eta, epsilon, f_sipm):
    """
    Returns the number of photons detected
    
    Inputs: dE_dx - energy deposition [keV/mm]
            v_d - drift velocity [mm/us]
            eta - gain of EL gap
            epsilon - efficiencies, 
            f_sipm - fraction of light to center most SiPM
    """
    return N_i(dE_dx) * v_d * eta * epsilon * f_sipm

## NEXT Flex

In [10]:
# Constants in all flex simulations 
epsilon_flex = .286
r_TP_flex = 984./2. # mm
d_EL_flex = 10. # mm
d_A_flex = 15. # mm
sipm_width = 2.
v_drift_flex = 1. # mm/us
P_flex = 15 # bar
Efield_flex = 16. # kV/cm
eta_flex = EL_yield(d_EL_flex, Efield_flex, P_flex) # photons/e
eta_flex_old = 500 #EL_yield_old(d_EL_flex, Efield_flex, P_flex)
C_pmts = 0.3 # need to check this
d_active_flex = 1160. # mm

In [11]:
# Simulation specific constants are included in the dictionary for that simulation
mcs = []
mcs.append({"size":1.3, "pitch":1.3, 'C':1.0, 'name':'1.3mm SiPM, 1.3mm pitch', "dir":"s1.3mmp1.3mm"})
mcs.append({"size":1.3, "pitch":7, 'C':0.034, 'name': '1.3mm SiPM, 7mm pitch',"dir": "s1.3mmp7mm"})
mcs.append({"size":1.3, "pitch":15, 'C':0.007, 'name': '1.3mm SiPM, 15mm pitch',"dir": "s1.3mmp15mm"})

mcs.append({"size":3, "pitch":3, 'C':1.0, 'name':'3mm SiPM, 3mm pitch', "dir":"s3mmp3mm"})
mcs.append({"size":3, "pitch":7, 'C':0.32, 'name': '3mm SiPM, 7mm pitch',"dir": "s3mmp7mm"})
mcs.append({"size":3, "pitch":15, 'C':0.07, 'name': '3mm SiPM, 15mm pitch', "dir": "s3mmp15mm"})

mcs.append({"size":6, "pitch":6, 'C':1.0, 'name':'6mm SiPM, 6mm pitch', "dir":"s6mmp6mm"})
mcs.append({"size":6, "pitch":15, 'C':0.28, 'name': '6mm SiPM, 15mm pitch', "dir": "s6mmp15mm"})

In [12]:
for mc in mcs:
    print(mc['name'])
    print('   Photons at TP = '+str(N_i(E_kr) * eta_flex * SAF_Plane(r_TP_flex, d_EL_flex, d_A_flex-sipm_width) * mc['C']))

1.3mm SiPM, 1.3mm pitch
   Photons at TP = 886599.0994925604
1.3mm SiPM, 7mm pitch
   Photons at TP = 30144.369382747056
1.3mm SiPM, 15mm pitch
   Photons at TP = 6206.193696447923
3mm SiPM, 3mm pitch
   Photons at TP = 886599.0994925604
3mm SiPM, 7mm pitch
   Photons at TP = 283711.7118376193
3mm SiPM, 15mm pitch
   Photons at TP = 62061.93696447923
6mm SiPM, 6mm pitch
   Photons at TP = 886599.0994925604
6mm SiPM, 15mm pitch
   Photons at TP = 248247.74785791693


In [13]:
print('-------------------------')
print('NEXT Flex')
print('SiPM Solid angle fractions')
mcs_bysize = [mcs[i] for i in [0,3,6]]
for mc in mcs_bysize:
    f_sipm_min = SAF_square(mc['size'], d_EL_flex + d_A_flex-sipm_width)
    f_sipm_max = SAF_square(mc['size'], d_A_flex-sipm_width)
    f_sipm_approx = SAF_square_approx(mc['size'], d_EL_flex + d_A_flex-sipm_width)
    print('-------------------------')
    print(str(mc['size'])+'mm SiPM: ')
    print('   f_SiPM range = '+str(f_sipm_min)+'-'+str(f_sipm_max))
    print('   f_SiPM(approx) = '+str(f_sipm_approx))
    print('   f_SiPM / f_SiPM(approx) = '+str(f_sipm_min / f_sipm_approx))
print('-------------------------')

-------------------------
NEXT Flex
SiPM Solid angle fractions
-------------------------
1.3mm SiPM: 
   f_SiPM range = 0.0002540238489818502-0.000793791062608091
   f_SiPM(approx) = 0.0002542267049388499
   f_SiPM / f_SiPM(approx) = 0.9992020666867059
-------------------------
3mm SiPM: 
   f_SiPM range = 0.0013481400189891217-0.004182294021107423
   f_SiPM(approx) = 0.0013538700263015673
   f_SiPM / f_SiPM(approx) = 0.9957676828638429
-------------------------
6mm SiPM: 
   f_SiPM range = 0.005325134961943662-0.016101185498196478
   f_SiPM(approx) = 0.005415480105206269
   f_SiPM / f_SiPM(approx) = 0.9833172421452067
-------------------------


In [14]:
print('-------------------------')
print('NEXT Flex')
print('dE/dx for new and old ELyields')
print('New ELyield = '+str(eta_flex))
print('Old ELyiled = '+str(eta_flex_old))
print('Ratio = '+str(eta_flex_old/eta_flex))
for mc in mcs:
    f_flex = SAF_Plane(r_TP_flex, d_EL_flex, d_A_flex-sipm_width) * mc['C']
    f_sipm = SAF_SiPM(mc['size'], d_EL_flex, d_A_flex-sipm_width)
    
    print('-------------------------')
    print(mc['name'])
    print('  Old ELyield:')
    print('    N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex_old, epsilon_flex, f_flex))))
    print('    N_d(Kr) = '+str(int(N_d(E_kr, eta_flex_old, epsilon_flex, f_flex))))
    print('    dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex_old, epsilon_flex, f_sipm))))
    print('  New ELyield:')
    print('    N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex, epsilon_flex, f_flex))))
    print('    N_d(Kr) = '+str(int(N_d(E_kr, eta_flex, epsilon_flex, f_flex))))
    print('    dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex, epsilon_flex, f_sipm))))
    print('  Ratios')
    print('    N_d(Qbb,old)/N_d(Qbb, new) = '+str(N_d(E_qbb, eta_flex_old, epsilon_flex, f_flex) / N_d(E_qbb, eta_flex, epsilon_flex, f_flex)))
    print('    N_d(kr, old)/N_d(kr, new) = '+str(N_d(E_kr, eta_flex_old, epsilon_flex, f_flex) / N_d(E_kr, eta_flex, epsilon_flex, f_flex)))
    print('    dN_dt(qbb, blob, old)/new = '+str(dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex_old, epsilon_flex, f_sipm) / dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex, epsilon_flex, f_sipm)))
print('-------------------------')

-------------------------
NEXT Flex
dE/dx for new and old ELyields
New ELyield = 980.0000000000002
Old ELyiled = 500
Ratio = 0.510204081632653
-------------------------
1.3mm SiPM, 1.3mm pitch
  Old ELyield:
    N_d(Qbb) = 7481701
    N_d(Kr) = 129371
    dN_dt(Qbb, blob) = 66
  New ELyield:
    N_d(Qbb) = 14664135
    N_d(Kr) = 253567
    dN_dt(Qbb, blob) = 131
  Ratios
    N_d(Qbb,old)/N_d(Qbb, new) = 0.510204081632653
    N_d(kr, old)/N_d(kr, new) = 0.5102040816326531
    dN_dt(qbb, blob, old)/new = 0.510204081632653
-------------------------
1.3mm SiPM, 7mm pitch
  Old ELyield:
    N_d(Qbb) = 254377
    N_d(Kr) = 4398
    dN_dt(Qbb, blob) = 66
  New ELyield:
    N_d(Qbb) = 498580
    N_d(Kr) = 8621
    dN_dt(Qbb, blob) = 131
  Ratios
    N_d(Qbb,old)/N_d(Qbb, new) = 0.510204081632653
    N_d(kr, old)/N_d(kr, new) = 0.510204081632653
    dN_dt(qbb, blob, old)/new = 0.510204081632653
-------------------------
1.3mm SiPM, 15mm pitch
  Old ELyield:
    N_d(Qbb) = 52371
    N_d(Kr) = 90

In [15]:
f_ep = SAF_Plane(r_TP_flex, d_active_flex, 0)
dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex, epsilon_flex, f_ep)

29706.207608737288

In [16]:
print('-------------------------')
print('NEXT Flex SiPMs')
for mc in mcs:
    f_flex = SAF_Plane(r_TP_flex, 0, d_A_flex-sipm_width) * mc['C']
    f_sipm = SAF_SiPM(mc['size'], 0, d_A_flex-sipm_width)
    print('-------------------------')
    print(mc['name']+': ')
    print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex, epsilon_flex, f_flex))))
    print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_flex, epsilon_flex, f_flex))))
    print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_flex, eta_flex, epsilon_flex, f_sipm))))
    print('   dN_dt(Qbb, track) = '+str(int(dN_dt(dEdx_qbb_track, v_drift_flex, eta_flex, epsilon_flex, f_sipm))))
print('-------------------------')

-------------------------
NEXT Flex SiPMs
-------------------------
1.3mm SiPM, 1.3mm pitch: 
   N_d(Qbb) = 14816648
   N_d(Kr) = 256204
   dN_dt(Qbb, blob) = 277
   dN_dt(Qbb, track) = 97
-------------------------
1.3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 503766
   N_d(Kr) = 8710
   dN_dt(Qbb, blob) = 277
   dN_dt(Qbb, track) = 97
-------------------------
1.3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 103716
   N_d(Kr) = 1793
   dN_dt(Qbb, blob) = 277
   dN_dt(Qbb, track) = 97
-------------------------
3mm SiPM, 3mm pitch: 
   N_d(Qbb) = 14816648
   N_d(Kr) = 256204
   dN_dt(Qbb, blob) = 1454
   dN_dt(Qbb, track) = 508
-------------------------
3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 4741327
   N_d(Kr) = 81985
   dN_dt(Qbb, blob) = 1454
   dN_dt(Qbb, track) = 508
-------------------------
3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 1037165
   N_d(Kr) = 17934
   dN_dt(Qbb, blob) = 1454
   dN_dt(Qbb, track) = 508
-------------------------
6mm SiPM, 6mm pitch: 
   N_d(Qbb) = 14816648
   N_d(Kr) = 256204
   dN

In [17]:
print('-------------------------')
print('NEXT Flex PMTs')
for mc in mcs:
    f_flex = SAF_Plane(r_TP_flex, d_EL_flex+d_active_flex, d_A_flex-sipm_width) * C_pmts
    print('-------------------------')
    print(mc['name']+': ')
    print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_flex, 1.0, f_flex))))
    print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_flex, 1.0, f_flex))))
print('-------------------------')

-------------------------
NEXT Flex PMTs
-------------------------
1.3mm SiPM, 1.3mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
1.3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
1.3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
3mm SiPM, 3mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
3mm SiPM, 7mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
3mm SiPM, 15mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
6mm SiPM, 6mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------
6mm SiPM, 15mm pitch: 
   N_d(Qbb) = 3641291
   N_d(Kr) = 62963
-------------------------


## DEMO

In [ ]:
# Constants in demo
eta_demo = 1100
epsilon_demo = .22 * .88 # effective detection efficiency * anode transparaency 
r_TP_demo = 300./2. # mm
d_EL_demo = 5. # mm
d_A_demo = 2. # mm
s_demo = 1. # mm
N_sipms_demo = 256
C_demo = Coverage(r_TP_demo, s_demo, N_sipms_demo)
f_demo = SAF_Plane(r_TP_demo, d_EL_demo, d_A_demo) * C_demo
f_sipm_demo = SAF_SiPM(s_demo, d_EL_demo, d_A_demo, 0)
v_drift_demo = 1.0

print('N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, epsilon_demo, f_demo))))
print('N_d(kr) = '+str(int(N_d(E_kr, eta_demo, epsilon_demo, f_demo))))

In [ ]:
f_sipm_min = SAF_square(s_demo, d_EL_demo + d_A_demo)
f_sipm_max = SAF_square(s_demo, d_A_demo)
f_sipm_approx = SAF_square_approx(s_demo, d_EL_demo + d_A_demo)
print('f_SiPM range = '+str(f_sipm_min)+'-'+str(f_sipm_max))
print('f_SiPM(approx) = '+str(f_sipm_approx))
print('f_SiPM / f_SiPM(approx) = '+str(f_sipm_min / f_sipm_approx))
print('f_SiPM_max / f_SiPM_min = '+str(f_sipm_max/f_sipm_min))
print('f_SiPM_mean / f_SiPM_min = '+str(f_sipm_demo/f_sipm_min))

In [ ]:
print('-------------------------')
print('DEMO SiPMs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, epsilon_demo, f_demo))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_demo, epsilon_demo, f_demo))))
print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_demo, eta_demo, epsilon_demo, f_sipm_demo))))
print('   dN_dt(Qbb, track) = '+str(int(dN_dt(dEdx_qbb_track, v_drift_demo, eta_demo, epsilon_demo, f_sipm_demo))))
print('   dN_dt(Qbb, track, approx like Javi) = '+str(int(dN_dt(dEdx_qbb_track, v_drift_demo, eta_demo, epsilon_demo, SAF_square_approx(s_demo, d_EL_demo+d_A_demo)))))
print('-------------------------')

In [ ]:
print('-------------------------')
print('DEMO PMTs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_demo, 1.0, f_demo))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_demo, 1.0, f_demo))))
print('-------------------------')

## NEW

SiPMs see about 22,000 pes total for a kr event. This was done roughly with run 7470, not cuts.

In [ ]:
# Constants in demo
P_new = 10. # bar 
Efield_new = 17. # kV/cm
epsilon_new = .07 # should edit this somehow
r_TP_new = 454./2. # mm
d_EL_new = 10. # mm
d_A_new = 2. # mm # to fix
eta_new = EL_yield(d_EL_new, Efield_new, P_new)
s_new = 1.3 # mm
N_sipms_new = 1792
C_new = Coverage(r_TP_new, s_new, N_sipms_new)
f_new = SAF_Plane(r_TP_new, d_EL_new, d_A_new) * C_new
f_sipm_new = SAF_SiPM(s_new, d_EL_new, d_A_new)
v_drift_new = 1. # mm/us
d_drift_new = 664.5 # mm

N_d_data = 22000 # roughly average total signal in SiPMs from kr events in NEW in run 7470
eff_new = N_d_data / (N_i(E_kr, W_i=21.9) * eta_new * f_new)

In [ ]:
eta_new

In [ ]:
eff_new

In [ ]:
print('-------------------------')
print('NEW SiPMs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_new, epsilon_new, f_new))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_new, epsilon_new, f_new))))
print('   dN_dt(Qbb, blob) = '+str(int(dN_dt(dEdx_qbb_blob, v_drift_new, eta_new, epsilon_new, f_sipm))))
print('-------------------------')

In [ ]:
print('-------------------------')
print('NEW PMTs')
print('   N_d(Qbb) = '+str(int(N_d(E_qbb, eta_new, 1.0, f_new))))
print('   N_d(Kr) = '+str(int(N_d(E_kr, eta_new, 1.0, f_new))))
print('-------------------------')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Testing eres stuff
names = ['s12mmp15', 's13mmp7mm','s3mmp15mm', 's6mmp15mm', 's3mmp7mm', 's3mmp3mm','s6mmp6mm']
coverages = [0.73514, 3.3930, 3.914965, 15.65986, 18.0694, 98.37593, 98.394868]
coverages = [1., 3., 5., 18., 20., 99., 100.]
nt_fwhm = [471.68,1541.88,2033.38,7321.10,8175.60, 44389.90, 49565.82]
nt_mean = [120187.23,553314.64,793155.23,3512105.58,3662840.31,20399213.63,22473237.79]

In [ ]:
plt.plot(coverages, nt_fwhm)

# Comparing with Detsim functions

In [ ]:
from invisible_cities.core import system_of_units as units
from typing import Tuple

In [ ]:
def generate_ionization_electrons(energies    : np.ndarray,
                                  wi          : float = 22.4 * units.eV,
                                  fano_factor : float = 0.15) -> np.ndarray:
    """ Generate secondary ionization electrons from energy deposits
    Parameters:
        :wi: float
            Mean ionization energy
        :fano_factor: float
            Fano-factor. related with the deviation in ionization electrons
        :energies: np.ndarray
            Energy hits
    Returns:
        :nes: np.ndarray
            The ionization electrons per hit
    Comment:
        The quotient energy/wi gives the mean value of the ionization electrons produced
        in a hit (N).
        The number ionization electrons produced in a hit (N) are assumed to be normally
        distributed with mean N and variance (N*F). (Being F the fano_factor).
        If the variance is such that <1, the electrons are assumed to be poisson distributed.
    """
    nes = energies / wi
    var = nes * fano_factor
    pois = var < 1      #See docstring for explanation
    nes[ pois] = np.random.poisson(nes[pois])
    nes[~pois] = np.round(np.random.normal(nes[~pois], np.sqrt(var[~pois])))
    nes[nes<0] = 0
    return nes.astype(int)

def drift_electrons(zs             : np.ndarray,
                    n_electrons    : np.ndarray,
                    lifetime       : float = 12 * units.ms,
                    drift_velocity : float = 1  * units.mm / units.mus) -> np.ndarray:
    """ Returns number of electrons due to lifetime losses from secondary electrons
    Parameters:
        :lifetime: float
            Electron lifetime
        :drift_velocity: float
            Drift velocity at the active volume of the detector
        :zs:
            z coordinate of the ionization hits
        :electrons:
            Number of ionization electrons in each hit
    Returns:
        :nes: np.ndarray
            Number of ionization electrons that reach the EL gap
    """
    @np.vectorize
    def attachment(n_ie, t):
        return np.count_nonzero(-lifetime * np.log(np.random.uniform(size=n_ie)) > t)

    ts  = zs / drift_velocity
    return attachment(n_electrons, ts)

def diffuse_electrons(xs                     : np.ndarray,
                      ys                     : np.ndarray,
                      zs                     : np.ndarray,
                      n_electrons            : np.ndarray,
                      transverse_diffusion   : float = 1.0 * units.mm / units.cm**0.5,
                      longitudinal_diffusion : float = 0.2 * units.mm / units.cm**0.5)\
                      -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Starting from hits with positions xs, ys, zs, and number of electrons,
    apply diffusion and return diffused positions xs, ys, zs for each electron.
    Paramters:
        :transverse_diffusion: float
        :longitudinal_diffusion: float
        :xs, ys, zs: np.ndarray (1D of size: #hits)
            Postions of initial hits
        :electrons:
            Number of ionization electrons per hit before drifting
    Returns:
        :dxs, dys, dzs: np.ndarray (1D of size: #hits x #electrons-per-hit)
            Diffused positions at the EL
    """
    xs = np.repeat(xs, n_electrons)
    ys = np.repeat(ys, n_electrons)
    zs = np.repeat(zs, n_electrons)

    # substitute z<0 to z=0
    sel = zs<0
    zs[sel] = 0

    sqrtz = zs ** 0.5
    dxs  = np.random.normal(xs, sqrtz *   transverse_diffusion)
    dys  = np.random.normal(ys, sqrtz *   transverse_diffusion)
    dzs  = np.random.normal(zs, sqrtz * longitudinal_diffusion)

    return (dxs, dys, dzs)

def ielectron_simulator(*, wi: float, fano_factor: float, lifetime: float,
                        transverse_diffusion: float, longitudinal_diffusion: float, drift_velocity:float,
                        el_gain: float, conde_policarpo_factor: float):
    """
    Function that simulates electron creation, drift, diffusion and photon generation at the EL
    Parameters: floats
        parameter names are self-descriptive.
    Returns:
        :simulate_ielectrons:
            function that returns the positions emission times and number of photons at the EL
    """
    def simulate_ielectrons(x, y, z, time, energy):
        nelectrons = generate_ionization_electrons(energy, wi, fano_factor)
        nelectrons = drift_electrons(z, nelectrons, lifetime, drift_velocity)
        dx, dy, dz = diffuse_electrons(x, y, z, nelectrons, transverse_diffusion, longitudinal_diffusion)
        dtimes = dz/drift_velocity + np.repeat(time, nelectrons)
        nphotons = np.random.normal(el_gain, np.sqrt(el_gain * conde_policarpo_factor), size=nelectrons.sum())
        nphotons = np.round(nphotons).astype(np.int32)
        return dx, dy, dz, dtimes, nphotons
    return simulate_ielectrons



In [ ]:
zs = np.random.uniform(0,1200,1000)
xs = np.random.uniform(0,500,1000)
ys = np.random.uniform(0,500,1000)
ts = np.repeat(1.0, 1000)
es = np.repeat(41.5, 1000) # keV

In [ ]:
plt.hist(xs)
plt.show()

plt.hist(ys)
plt.show()

plt.hist(es)
plt.show()

In [ ]:
detsim_nphotons = ielectron_simulator(wi=21.9, fano_factor=0.15, lifetime=1000, 
                                      transverse_diffusion=0.0, longitudinal_diffusion=0.0, 
                                      drift_velocity=1.0, el_gain=980, conde_policarpo_factor=1.0)


In [ ]:
_, _, _, _, nphotons = detsim_nphotons(xs,ys,zs, zs, es)

In [ ]:
plt.hist(nphotons)

In [ ]:
np.mean(nphotons)

In [ ]:
N_i(41.5), np.mean(generate_ionization_electrons(np.repeat([41.5/1000.],1000)))

# Checking PSF Values

In [ ]:
s13 = SAF_square(1.3, d_EL_flex + d_A_flex-sipm_width)
s3 = SAF_square(3, d_EL_flex + d_A_flex-sipm_width)
s6 = SAF_square(6, d_EL_flex + d_A_flex-sipm_width)

In [ ]:
# Simulation specific constants are included in the dictionary for that simulation
mcs = []
mcs.append({"size":1.3, "pitch":1.3, 'C':1.0, 'name':'1.3mm SiPM, 1.3mm pitch', "dir":"s1.3mmp1.3mm", "psf0":0.00011871001215240151, "psf300":1.2470967623013107e-07})
mcs.append({"size":1.3, "pitch":7., 'C':0.034, 'name': '1.3mm SiPM, 7mm pitch',"dir": "s1.3mmp7mm", "psf0":0.00011856003388343188, "psf300":1.1494174531864561e-07})
mcs.append({"size":1.3, "pitch":15., 'C':0.007, 'name': '1.3mm SiPM, 15mm pitch',"dir": "s1.3mmp15mm", "psf0":0.00011893361389618711, "psf300":1.146742916606954e-07})

mcs.append({"size":3., "pitch":3., 'C':1.0, 'name':'3mm SiPM, 3mm pitch', "dir":"s3mmp3mm", "psf0":0.0007919782296415871, "psf300":6.549495756436136e-07})
mcs.append({"size":3., "pitch":7., 'C':0.32, 'name': '3mm SiPM, 7mm pitch',"dir": "s3mmp7mm", "psf0":0.0008247988368565524, "psf300":6.801043281436533e-07})
mcs.append({"size":3., "pitch":15., 'C':0.07, 'name': '3mm SiPM, 15mm pitch', "dir": "s3mmp15mm", "psf0":0.0008261087191711004, "psf300":6.723618271161723e-07})

mcs.append({"size":6., "pitch":6., 'C':1.0, 'name':'6mm SiPM, 6mm pitch', "dir":"s6mmp6mm", "psf0":0.00365791515029429, "psf300":3.009927449907834e-06})
mcs.append({"size":6., "pitch":15., 'C':0.28, 'name': '6mm SiPM, 15mm pitch', "dir": "s6mmp15mm", "psf0":0.0036578596110938173, "psf300":2.8116593176238095e-06})

In [ ]:
# Simulation specific constants are included in the dictionary for that simulation
mcs_teflon = []
mcs_teflon.append({"size":1.3, "pitch":7., 'C':0.034, 'name': '1.3mm SiPM, 7mm pitch',"dir": "s1.3mmp7mm", "psf0":0.0001185036876623839, "psf300":1.692104419689629e-07})
mcs_teflon.append({"size":1.3, "pitch":15., 'C':0.007, 'name': '1.3mm SiPM, 15mm pitch',"dir": "s1.3mmp15mm", "psf0":0.00011884015628012157, "psf300":1.916884227450357e-07})

mcs_teflon.append({"size":3., "pitch":7., 'C':0.32, 'name': '3mm SiPM, 7mm pitch',"dir": "s3mmp7mm", "psf0":0.000827469692453882, "psf300":9.675857892767514e-07})
mcs_teflon.append({"size":3., "pitch":15., 'C':0.07, 'name': '3mm SiPM, 15mm pitch', "dir": "s3mmp15mm", "psf0":0.0008256528548501207, "psf300":1.104158323637117e-06})

mcs_teflon.append({"size":6., "pitch":15., 'C':0.28, 'name': '6mm SiPM, 15mm pitch', "dir": "s6mmp15mm", "psf0":0.003659652481641934, "psf300":4.2901024227073796e-06})

In [ ]:
for mc in mcs:
    mc['sipm_saf'] = SAF_square_approx(mc['size'], d_EL_flex + d_A_flex-sipm_width)
for mc in mcs_teflon:
    mc['sipm_saf'] = SAF_square_approx(mc['size'], d_EL_flex + d_A_flex-sipm_width)

In [ ]:
for mc in mcs:
    for mc_teflon in mcs_teflon:
        if mc['size'] == mc_teflon['size'] and mc['pitch'] == mc_teflon['pitch']:
            print(mc['name'])
            print('    PSF(d=0) no teflon / teflon = '+str(mc['psf0']/mc_teflon['psf0']))
            print('    PSF(d=300) no teflon / teflon = '+str(mc['psf300']/mc_teflon['psf300']))

In [ ]:
mcs_bysize = {1.3:[], 3.0:[], 6.0:[]}
for mc in mcs:
    mcs_bysize[mc['size']].append(mc)
mcs_bysize_t = {1.3:[], 3.0:[], 6.0:[]}
for mc in mcs_teflon:
    mcs_bysize_t[mc['size']].append(mc)
    
mcs_bypitch = {1.3:[], 3.0:[], 6.0:[], 7.0:[], 15.0:[]}
for mc in mcs:
    mcs_bypitch[mc['pitch']].append(mc)
mcs_bypitch_t = {1.3:[], 3.0:[], 6.0:[], 7.0:[], 15.0:[]}
for mc in mcs_teflon:
    mcs_bypitch_t[mc['pitch']].append(mc)

In [ ]:
for pitchs in mcs_bypitch:
    print(str(pitchs) + 'mm pitch')
    for i in range(len(mcs_bypitch[pitchs])):
        for j in range(len(mcs_bypitch[pitchs])):
            if i != j:
                print('   PSF(d=0) '+ str(mcs_bypitch[pitchs][i]['size']) + '/'+ str(mcs_bypitch[pitchs][j]['size']) + ' = ' +
                      str(mcs_bypitch[pitchs][i]['psf0']/mcs_bypitch[pitchs][j]['psf0']))
                print('   SiPM SAF '+ str(mcs_bypitch[pitchs][i]['size']) + '/'+ str(mcs_bypitch[pitchs][j]['size']) + ' = ' +
                      str(mcs_bypitch[pitchs][i]['sipm_saf']/mcs_bypitch[pitchs][j]['sipm_saf']))

In [ ]:
for sizes in mcs_bysize:
    print(str(sizes) + 'mm size')
    done = []
    for i in range(len(mcs_bysize[sizes])):
        done.append(i)
        for j in range(len(mcs_bysize[sizes])):
            if i != j and j not in done:
                print('   PSF(d=0) '+ str(mcs_bysize[sizes][i]['pitch']) + '/'+ str(mcs_bysize[sizes][j]['pitch']) + ' = ' +
                      str(mcs_bysize[sizes][i]['psf0']/mcs_bysize[sizes][j]['psf0']))
                print('   PSF(d=300) '+ str(mcs_bysize[sizes][i]['pitch']) + '/'+ str(mcs_bysize[sizes][j]['pitch']) + ' = ' +
                      str(mcs_bysize[sizes][i]['psf300']/mcs_bysize[sizes][j]['psf300']))

In [ ]:
for mc in mcs:
    print(mc['name']+': PSF(d=0) / SiPM SAF = '+str(mc['psf0']/mc['sipm_saf']))